In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [2]:
df_wontfix=pd.read_csv("C:/Users/jaish/Downloads/SoftwareEngineering/Project_dataset/wontfix-issues.csv")
df_non_wontfix=pd.read_csv("C:/Users/jaish/Downloads/SoftwareEngineering/Project_dataset/non-wontfix-issues.csv")

In [3]:
df_wontfix.head(10)

,project_name,project_url,project_language,issue_url,issue_labels,nCommentsT,nActorsT,issue_labels_additional_to_wontfix,date_first_comment,issue_closing_date,date_last_comment,timeToCloseIssue,timeToDiscussIssue,TitleIssue,DescriptionIssue,AllCommentsIssue,meanCommentSize
0,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,6,2,NaN,14.06.18 14:07,14.06.18 14:41,15.06.18 11:31,0.023565,0.891493,"improve ""Enter"" key <U+00B7> ""","When I press the ""enter"" key and th...","When I press the ""enter"" key and th...",388.833333
1,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,missing api;wontfix,4,2,missing api;,01.06.18 11:01,01.06.18 11:21,04.06.18 12:11,0.013646,3.048634,Syntax renders whitespace character as part of...,When using the setting <code>edito...,When using the setting <code>edito...,244.500000
2,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,3,2,NaN,05.04.18 15:45,27.04.18 19:07,27.04.18 19:07,22.140289,22.140289,Archive only if sub-tasks are completed <U+00B...,"When I press archive, I only want t...","When I press archive, I only want t...",1003.666667
3,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,6,3,NaN,03.04.18 08:16,27.04.18 19:09,27.04.18 19:09,24.453032,24.453032,[Feature Request] auto save when change todo s...,I thinks it's fair feature that whe...,I thinks it's fair feature that whe...,263.500000
4,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,enhancement;wontfix,3,2,enhancement;,27.12.17 00:08,20.05.18 20:29,12.06.18 13:44,144.806007,167.524815,Maintain the project structure (indentation) a...,<strong>AS IS</strong>: Given a to...,<strong>AS IS</strong>: Given a to...,1266.000000
5,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,C#,https://github.com/zzzprojects/GraphDiff/issue...,wontfix,5,2,NaN,30.08.13 12:33,11.11.13 22:57,08.09.14 23:54,73.475185,374.472697,Null values in entity should not be passed to ...,In a distributed scenario we often ...,In a distributed scenario we often ...,313.400000
6,yysun/Git-Source-Control-Provider,https://github.com/yysun/Git-Source-Control-Pr...,C#,https://github.com/yysun/Git-Source-Control-Pr...,status:wontfix,2,2,status:,08.02.12 17:44,09.02.13 03:59,09.02.13 03:59,366.427199,366.427199,Support Visual Studio 2008 Database Project <...,"""Database Projects"" in Visual Studi...","""Database Projects"" in Visual Studi...",103.000000
7,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,wontfix,3,2,NaN,07.11.17 12:07,08.11.17 19:13,08.11.17 19:14,1.296111,1.296204,Can I use this with ASP.net core2.0 projects <...,Got a error saying that it was rest...,Got a error saying that it was rest...,116.333333
8,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,question;wontfix,2,2,question;,04.01.17 02:22,04.01.17 15:15,04.01.17 15:15,0.537049,0.537130,"MvcReportViewer Browser issue <U+00B7> """,About MvcReportViewer supported Man...,About MvcReportViewer supported Man...,265.500000
9,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,question;wontfix,6,6,question;,21.11.14 12:08,21.11.14 15:09,10.11.17 20:51,0.125938,1085.363345,"Scroll Bar issues in ie 11, works good in othe...",Scroll bar appears perfectly well i...,Scroll bar appears perfectly well i...,533.000000


In [4]:
df_non_wontfix.head

<bound method NDFrame.head of                 project_name                                project_url  \
0      zzzprojects/GraphDiff   https://github.com/zzzprojects/GraphDiff   
1      zzzprojects/GraphDiff   https://github.com/zzzprojects/GraphDiff   
2      zzzprojects/GraphDiff   https://github.com/zzzprojects/GraphDiff   
3      zzzprojects/GraphDiff   https://github.com/zzzprojects/GraphDiff   
4      zzzprojects/GraphDiff   https://github.com/zzzprojects/GraphDiff   
...                      ...                                        ...   
4481  nunit/nunit-vs-adapter  https://github.com/nunit/nunit-vs-adapter   
4482  nunit/nunit-vs-adapter  https://github.com/nunit/nunit-vs-adapter   
4483  nunit/nunit-vs-adapter  https://github.com/nunit/nunit-vs-adapter   
4484  nunit/nunit-vs-adapter  https://github.com/nunit/nunit-vs-adapter   
4485   zzzprojects/GraphDiff   https://github.com/zzzprojects/GraphDiff   

                                              issue_url  \
0     http

In [5]:
df_wontfix_temp = df_wontfix.drop(columns=['nCommentsT','nActorsT','issue_labels_additional_to_wontfix','date_first_comment','issue_closing_date','date_last_comment','timeToCloseIssue','timeToDiscussIssue','AllCommentsIssue','meanCommentSize'])

In [6]:
df_wontfix_temp.head(10)

,project_name,project_url,project_language,issue_url,issue_labels,TitleIssue,DescriptionIssue
0,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,"improve ""Enter"" key <U+00B7> ""","When I press the ""enter"" key and th..."
1,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,missing api;wontfix,Syntax renders whitespace character as part of...,When using the setting <code>edito...
2,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,Archive only if sub-tasks are completed <U+00B...,"When I press archive, I only want t..."
3,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,[Feature Request] auto save when change todo s...,I thinks it's fair feature that whe...
4,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,enhancement;wontfix,Maintain the project structure (indentation) a...,<strong>AS IS</strong>: Given a to...
5,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,C#,https://github.com/zzzprojects/GraphDiff/issue...,wontfix,Null values in entity should not be passed to ...,In a distributed scenario we often ...
6,yysun/Git-Source-Control-Provider,https://github.com/yysun/Git-Source-Control-Pr...,C#,https://github.com/yysun/Git-Source-Control-Pr...,status:wontfix,Support Visual Studio 2008 Database Project <...,"""Database Projects"" in Visual Studi..."
7,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,wontfix,Can I use this with ASP.net core2.0 projects <...,Got a error saying that it was rest...
8,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,question;wontfix,"MvcReportViewer Browser issue <U+00B7> """,About MvcReportViewer supported Man...
9,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,question;wontfix,"Scroll Bar issues in ie 11, works good in othe...",Scroll bar appears perfectly well i...


In [7]:
df_wontfix_temp['issue_labels'] = 'wontfix'

In [8]:
df_non_wontfix_temp = df_non_wontfix.copy()

In [9]:
df_non_wontfix_temp.head(10)

,project_name,project_url,issue_url,TitleIssue,DescriptionIssue,project_language
0,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issue...,"Recursive Many-to-Many Issue <U+00B7> """,I have an issue with a Recursive Ma...,C#
1,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issue...,Possible bug when the same entity is to be upd...,"Hi, updating an association-heavy ...",C#
2,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issue...,System.InvalidCastException in UpdateGraph whe...,I have a class Mission with a prope...,C#
3,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issue...,"Abstract class inheritance <U+00B7> ""","When Entity classes, for example Po...",C#
4,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issue...,"UpdateConfigurationVisitor fix <U+00B7> ""","Currently, if the root.IncludeStrin...",C#
5,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issues/5,Unable to cast object of type &#39;System.Linq...,Description: An unhandled exception...,C#
6,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issues/3,Can&#39;t save new entity when its parent prop...,I originally posted this on your bl...,C#
7,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff/issues/2,"Three Unit Tests Failing <U+00B7> """,AssociatedEntityValuesShouldNotBeUp...,C#
8,yysun/Git-Source-Control-Provider,https://github.com/yysun/Git-Source-Control-Pr...,https://github.com/yysun/Git-Source-Control-Pr...,"Diff margin can crash Visual Studio <U+00B7> """,Call stack: <pre><code>Microsoft.Vi...,C#
9,yysun/Git-Source-Control-Provider,https://github.com/yysun/Git-Source-Control-Pr...,https://github.com/yysun/Git-Source-Control-Pr...,"Diff margin causes slow scrolling <U+00B7> """,Profiling indicates that the primar...,C#


In [10]:
df_non_wontfix_temp['issue_labels'] = 'non wontfix'

In [11]:
merged_temp = [df_wontfix_temp, df_non_wontfix_temp]
df_final = pd.concat(merged_temp)
df_final.head(10)

,project_name,project_url,project_language,issue_url,issue_labels,TitleIssue,DescriptionIssue
0,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,"improve ""Enter"" key <U+00B7> ""","When I press the ""enter"" key and th..."
1,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,Syntax renders whitespace character as part of...,When using the setting <code>edito...
2,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,Archive only if sub-tasks are completed <U+00B...,"When I press archive, I only want t..."
3,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,[Feature Request] auto save when change todo s...,I thinks it's fair feature that whe...
4,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,Maintain the project structure (indentation) a...,<strong>AS IS</strong>: Given a to...
5,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,C#,https://github.com/zzzprojects/GraphDiff/issue...,wontfix,Null values in entity should not be passed to ...,In a distributed scenario we often ...
6,yysun/Git-Source-Control-Provider,https://github.com/yysun/Git-Source-Control-Pr...,C#,https://github.com/yysun/Git-Source-Control-Pr...,wontfix,Support Visual Studio 2008 Database Project <...,"""Database Projects"" in Visual Studi..."
7,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,wontfix,Can I use this with ASP.net core2.0 projects <...,Got a error saying that it was rest...
8,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,wontfix,"MvcReportViewer Browser issue <U+00B7> """,About MvcReportViewer supported Man...
9,ilich/MvcReportViewer,https://github.com/ilich/MvcReportViewer,C#,https://github.com/ilich/MvcReportViewer/issue...,wontfix,"Scroll Bar issues in ie 11, works good in othe...",Scroll bar appears perfectly well i...


In [12]:
def issue_number(i):
  if i['issue_labels'] == 'non wontfix':
    return 1
  else :
    return 0
df_final['issue_number'] = df_final.apply(lambda i:issue_number(i), axis=1)

In [13]:
df_final

,project_name,project_url,project_language,issue_url,issue_labels,TitleIssue,DescriptionIssue,issue_number
0,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,"improve ""Enter"" key <U+00B7> ""","When I press the ""enter"" key and th...",0
1,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,Syntax renders whitespace character as part of...,When using the setting <code>edito...,0
2,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,Archive only if sub-tasks are completed <U+00B...,"When I press archive, I only want t...",0
3,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,[Feature Request] auto save when change todo s...,I thinks it's fair feature that whe...,0
4,fabiospampinato/vscode-todo-plus,https://github.com/fabiospampinato/vscode-todo...,C#,https://github.com/fabiospampinato/vscode-todo...,wontfix,Maintain the project structure (indentation) a...,<strong>AS IS</strong>: Given a to...,0
...,...,...,...,...,...,...,...,...
4481,nunit/nunit-vs-adapter,https://github.com/nunit/nunit-vs-adapter,C#,https://github.com/nunit/nunit-vs-adapter/issu...,non wontfix,"Feature request : enable diagnostic logging · """,Some of the tests have been hanging on the bu...,1
4482,nunit/nunit-vs-adapter,https://github.com/nunit/nunit-vs-adapter,C#,https://github.com/nunit/nunit-vs-adapter/issu...,non wontfix,"Test Adapter does not find addins · ""","<a rel=""nofollow"" href=""https://groups.google...",1
4483,nunit/nunit-vs-adapter,https://github.com/nunit/nunit-vs-adapter,C#,https://github.com/nunit/nunit-vs-adapter/issu...,non wontfix,"Unable to disable shadow copy · """,The simple work around we've been using in NU...,1
4484,nunit/nunit-vs-adapter,https://github.com/nunit/nunit-vs-adapter,C#,https://github.com/nunit/nunit-vs-adapter/issu...,non wontfix,"Test Results · ""","Not only I need this. See <a rel=""nofollow"" h...",1


In [14]:
df_final.TitleIssue.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
4481    False
4482    False
4483     True
4484    False
4485     True
Name: TitleIssue, Length: 6330, dtype: bool

In [15]:
# Dropping duplicate Issue Titles
df_final.drop_duplicates(subset = ['TitleIssue'], inplace = True)

In [16]:
df_final.head

<bound method NDFrame.head of                           project_name  \
0     fabiospampinato/vscode-todo-plus   
1     fabiospampinato/vscode-todo-plus   
2     fabiospampinato/vscode-todo-plus   
3     fabiospampinato/vscode-todo-plus   
4     fabiospampinato/vscode-todo-plus   
...                                ...   
4479            nunit/nunit-vs-adapter   
4480            nunit/nunit-vs-adapter   
4481            nunit/nunit-vs-adapter   
4482            nunit/nunit-vs-adapter   
4484            nunit/nunit-vs-adapter   

                                            project_url project_language  \
0     https://github.com/fabiospampinato/vscode-todo...               C#   
1     https://github.com/fabiospampinato/vscode-todo...               C#   
2     https://github.com/fabiospampinato/vscode-todo...               C#   
3     https://github.com/fabiospampinato/vscode-todo...               C#   
4     https://github.com/fabiospampinato/vscode-todo...               C#   
...      

In [17]:
df_final_shuffled = df_final.sample(frac = 1)

In [18]:
df_final_shuffled.dropna(axis = 0, inplace = True)


In [19]:
df_final_shuffled

,project_name,project_url,project_language,issue_url,issue_labels,TitleIssue,DescriptionIssue,issue_number
1135,DotNetAnalyzers/StyleCopAnalyzers,https://github.com/DotNetAnalyzers/StyleCopAna...,C#,https://github.com/DotNetAnalyzers/StyleCopAna...,wontfix,SA1631: DocumentationTextMustMeetCharacterPerc...,"<a href=""http://www.stylecop.com/do...",0
3,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,C#,https://github.com/zzzprojects/GraphDiff/issue...,non wontfix,"Abstract class inheritance <U+00B7> ""","When Entity classes, for example Po...",1
2508,uncled1023/IRCBot,https://github.com/uncled1023/IRCBot,C#,https://github.com/uncled1023/IRCBot/issues/515,non wontfix,[VenomIreland] make windows 9 mandatory for ev...,<em>No description provided.</em> ...,1
1186,zzzprojects/GraphDiff,https://github.com/zzzprojects/GraphDiff,C#,https://github.com/zzzprojects/GraphDiff/issue...,non wontfix,"Using GraphDiff for updating entities · """,I can update my domain entities in separate w...,1
135,stormpath/stormpath-sdk-dotnet,https://github.com/stormpath/stormpath-sdk-dotnet,C#,https://github.com/stormpath/stormpath-sdk-dot...,wontfix,"Remove Action&lt;T&gt; overloads <U+00B7> """,The method overloads which take an ...,0
...,...,...,...,...,...,...,...,...
3080,denikson/CM3D2.MaidFiddler,https://github.com/denikson/CM3D2.MaidFiddler,C#,https://github.com/denikson/CM3D2.MaidFiddler/...,non wontfix,"do not use in x86 sybaris · """,</td> not use in x86 sybaris,1
589,xin9le/DeclarativeSql,https://github.com/xin9le/DeclarativeSql,C#,https://github.com/xin9le/DeclarativeSql/issue...,non wontfix,<U+7279><U+5B9A><U+306E>Update<U+3067><U+30A8>...,<U+8868><U+984C><U+306E><U+4EF6><U+...,1
311,adamralph-archive/scriptcs-nancy,https://github.com/adamralph-archive/scriptcs-...,C#,https://github.com/adamralph-archive/scriptcs-...,non wontfix,Move module finding into default bootstrapper ...,"<p class=""text-gray""> <em...",1
3310,codebude/QRCoder,https://github.com/codebude/QRCoder,C#,https://github.com/codebude/QRCoder/issues/175,non wontfix,"need QR code decoder C# code · """,<h2>Type of issue</h2> [ ] Bug<br>[ ] Question...,1


In [20]:
temp_drop = df_final_shuffled[['issue_labels', 'issue_number']].drop_duplicates()

In [21]:

final_class = dict(temp_drop[['issue_number', 'issue_labels']].values)


In [22]:
final_class

{0: 'wontfix', 1: 'non wontfix'}

DATA PREPROCESSING

In [25]:
import nltk
from nltk.corpus import stopwords

In [26]:
import re
def remove_HTML(string):
  tags = re.sub('<.*?>','',str(string))
  return tags

df_final_shuffled['TitleIssue'] = df_final_shuffled['TitleIssue'].apply(lambda x : remove_HTML(x))
df_final_shuffled['DescriptionIssue'] = df_final_shuffled['DescriptionIssue'].apply(lambda x : remove_HTML(x))

In [27]:
from nltk.tokenize import word_tokenize

In [28]:
def tokenize_text(text_string):
    return word_tokenize(text_string)

df_final_shuffled['TitleIssue'] = df_final_shuffled['TitleIssue'].apply(lambda x : tokenize_text(x))
df_final_shuffled['DescriptionIssue'] = df_final_shuffled['DescriptionIssue'].apply(lambda x : tokenize_text(x))

In [29]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer

#defining the object for stemming
porter_stemmer = PorterStemmer()

In [30]:
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

In [31]:
#applying the function
df_final_shuffled['TitleIssue'] = df_final_shuffled['TitleIssue'].apply(lambda x : stemming(x))
df_final_shuffled['DescriptionIssue'] = df_final_shuffled['DescriptionIssue'].apply(lambda x : stemming(x))

In [32]:
#importing nlp library
import nltk

stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [33]:
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [34]:
#applying the function
df_final_shuffled['TitleIssue'] = df_final_shuffled['TitleIssue'].apply(lambda x : remove_stopwords(x))
df_final_shuffled['DescriptionIssue'] = df_final_shuffled['DescriptionIssue'].apply(lambda x : remove_stopwords(x))

In [35]:
def convert_to_lowercase(text_string):
    return str(text_string).lower()

In [36]:
#applying the function
df_final_shuffled['TitleIssue'] = df_final_shuffled['TitleIssue'].apply(convert_to_lowercase)
df_final_shuffled['DescriptionIssue'] = df_final_shuffled['DescriptionIssue'].apply(convert_to_lowercase)

CLASSIFICATION

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
issuetitle_tf= TfidfVectorizer(min_df=5, stop_words='english')

In [39]:
issuedes_tf= TfidfVectorizer(min_df=5, stop_words='english')

In [40]:
tokenized_text_features31 = list(issuetitle_tf.fit_transform(df_final_shuffled['TitleIssue']).toarray())

In [41]:
tokenized_text_features32 = list(issuedes_tf.fit_transform(df_final_shuffled['DescriptionIssue']).toarray())

In [42]:
features_X = []
for i,j in enumerate(tokenized_text_features31):
  temp1 = np.append(tokenized_text_features31[i],tokenized_text_features32[i])
  #temp2 = np.append(y[i],temp1)
  features_X.append(temp1)
features_X = np.array(features_X)


In [43]:
from sklearn.model_selection import train_test_split
X_train_31, X_test_31, Y_train_31, Y_test_31 = train_test_split(features_X, df_final_shuffled['issue_number'].values, test_size=0.5, random_state = 400)

In [44]:
mn = MultinomialNB()
y_pred_31 = mn.fit(X_train_31, Y_train_31).predict(X_test_31)
print("Naive Bayes - ", accuracy_score(Y_test_31, y_pred_31))

Naive Bayes -  0.760427438814202


In [45]:
print('accuracy %.2f' % (accuracy_score(y_pred_31, Y_test_31)*100) + '%')
print(classification_report(Y_test_31, y_pred_31))
mn = confusion_matrix(Y_test_31, y_pred_31)
print(confusion_matrix(Y_test_31, y_pred_31))
#sn.heatmap(mn, annot=True)

accuracy 76.04%
              precision    recall  f1-score   support

           0       0.73      0.39      0.51       923
           1       0.77      0.93      0.84      1978

    accuracy                           0.76      2901
   macro avg       0.75      0.66      0.68      2901
weighted avg       0.75      0.76      0.74      2901

[[ 362  561]
 [ 134 1844]]


In [46]:
lr = LogisticRegression()
y_pred_32 = lr.fit(X_train_31, Y_train_31).predict(X_test_31)
print("Logistic Regression - ", accuracy_score(Y_test_31, y_pred_32))

Logistic Regression -  0.7552568079972424


In [47]:
print('accuracy %.2f' % (accuracy_score(y_pred_32, Y_test_31)*100) + '%')
print(classification_report(Y_test_31, y_pred_32))
lr = confusion_matrix(Y_test_31, y_pred_32)
print(confusion_matrix(Y_test_31, y_pred_32))

accuracy 75.53%
              precision    recall  f1-score   support

           0       0.73      0.37      0.49       923
           1       0.76      0.94      0.84      1978

    accuracy                           0.76      2901
   macro avg       0.74      0.65      0.66      2901
weighted avg       0.75      0.76      0.73      2901

[[ 340  583]
 [ 127 1851]]


In [48]:
from sklearn.tree import DecisionTreeClassifier

In [49]:
j48 = DecisionTreeClassifier(criterion="entropy", max_depth=3, random_state = 400)
#j48 = DecisionTreeClassifier(criterion="entropy", max_depth=3, random_state = 100)
y_pred_33 = j48.fit(X_train_31, Y_train_31).predict(X_test_31)
print("J48 - ", accuracy_score(Y_test_31, y_pred_33))

J48 -  0.712857635298173


In [50]:
print('accuracy %.2f' % (accuracy_score(y_pred_33, Y_test_31)*100) + '%')
print(classification_report(Y_test_31, y_pred_33))
j48 = confusion_matrix(Y_test_31, y_pred_33)
print(confusion_matrix(Y_test_31, y_pred_33))
#sn.heatmap(mn, annot=True)

accuracy 71.29%
              precision    recall  f1-score   support

           0       0.96      0.10      0.18       923
           1       0.70      1.00      0.83      1978

    accuracy                           0.71      2901
   macro avg       0.83      0.55      0.50      2901
weighted avg       0.79      0.71      0.62      2901

[[  94  829]
 [   4 1974]]


In [51]:
rf = RandomForestClassifier()
y_pred_34 = rf.fit(X_train_31, Y_train_31).predict(X_test_31)
print("Random Forest Classifier - ", accuracy_score(Y_test_31, y_pred_34))

Random Forest Classifier -  0.7521544295070666


In [52]:
print('accuracy %.2f' % (accuracy_score(y_pred_34, Y_test_31)*100) + '%')
print(classification_report(Y_test_31, y_pred_34))
rf = confusion_matrix(Y_test_31, y_pred_34)
print(confusion_matrix(Y_test_31, y_pred_34))
#sn.heatmap(mn, annot=True)

accuracy 75.22%
              precision    recall  f1-score   support

           0       0.74      0.34      0.47       923
           1       0.75      0.94      0.84      1978

    accuracy                           0.75      2901
   macro avg       0.75      0.64      0.65      2901
weighted avg       0.75      0.75      0.72      2901

[[ 315  608]
 [ 111 1867]]
